In [5]:
from collections import Counter
import gc
gc.enable()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.python.data import Dataset
tf.logging.set_verbosity(tf.logging.ERROR)


# Okay, let's load in our data_sets
raw_train_df = pd.read_csv("/home/wasi/Desktop/scrapy_projects/DL_Projects/Titanic_folder/train.csv")
raw_test_df = pd.read_csv("/home/wasi/Desktop/scrapy_projects/DL_Projects/Titanic_folder/test.csv")
example_submission_df = pd.read_csv("/home/wasi/Desktop/scrapy_projects/DL_Projects/Titanic_folder/"
                                    "gender_submission.csv")

train_df = raw_train_df.copy(deep=True)
test_df = raw_test_df.copy(deep=True)
train_test_lst = [train_df, test_df]
train_df[train_df['Fare'] == 0]

# Both train and test datasets appear to have NaN values this could cause problems for our model, so let's
# look at what is missing and how much

# print(train_df.isnull().sum())
# print("Total indiviuals in train set is : {}".format(len(train_df)))
#
# print(test_df.isnull().sum())
# print("Total individuals in test set is: {}".format(len(test_df)))

# The huge amount of missing Cabin data is worrying, but let's see if it has any predictive power
#  before figuring out what to do

cabin_df = train_df[train_df['Cabin'].notnull()]
cabin_df = cabin_df.assign(deck_level=pd.Series([entry[:1] for entry in cabin_df['Cabin']]).values)
print(cabin_df.head())
print("Survival chances based on deck level:")
print(round(cabin_df.groupby(['deck_level'])['Survived'].mean() * 100))


def process_deck_lst(train_test_lst):
    new = []
    for dataset in train_test_lst:
        dataset = dataset.copy(True)
        dataset = dataset.assign(deck_level=pd.Series([[entry[:1]] if not pd.isnull(entry)
                                                       else
                                                       'U'
                                                       for entry in dataset['Cabin']]))
        dataset = dataset.drop(['Cabin'], axis=1)
        new.append(dataset)
    return new


train_df, test_df = process_deck_lst(train_test_lst)
# print(train_df)
# print(test_df)
# print(train_df.isnull().sum())
# print(test_df.isnull().sum())

    PassengerId  Survived  Pclass  \
1             2         1       1   
3             4         1       1   
6             7         0       1   
10           11         1       3   
11           12         1       1   

                                                 Name     Sex   Age  SibSp  \
1   Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
6                             McCarthy, Mr. Timothy J    male  54.0      0   
10                    Sandstrom, Miss. Marguerite Rut  female   4.0      1   
11                           Bonnell, Miss. Elizabeth  female  58.0      0   

    Parch    Ticket     Fare Cabin Embarked deck_level  
1       0  PC 17599  71.2833   C85        C          C  
3       0    113803  53.1000  C123        S          C  
6       0     17463  51.8625   E46        S          E  
10      1   PP 9549  16.7000    G6        S          G  
11      0    113783  26

In [6]:
train_df.groupby(['Pclass', 'deck_level']).size().unstack(0).plot.bar(stacked=True)
plt.title("Histogram of deck_level grouped by Pclass")
_ = plt.ylabel("Frequency")

TypeError: unhashable type: 'list'